<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/Spark_SQL_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pyspark and customize Colab configuration


The pyspark installation will persist until the runtime is recycled.

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
G

'/usr/local/lib/python3.10/dist-packages/pyspark'

# Initialize Spark

In [2]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

sc = spark.sparkContext
spark

# Load Cars Dataset

In [3]:
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv

--2024-10-27 16:56:29--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv [following]
--2024-10-27 16:56:29--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22608 (22K) [text/csv]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  22.08K  --.-KB/s    in 0s      

2024-10-27 16:56:29 (170 MB/s) - ‘cars.csv’ saved [22608/22608]



In [4]:
# Using Apache Spark to read the CSV file into a DataFrame
df = df = spark.read.csv('cars.csv', header=True, sep=";")
df.createOrReplaceTempView("cars")

In [5]:
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0| 4354.|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|


# Data Types

In [6]:
spark.sql("DESC cars").show()

# This dataset has all string attributes

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|         Car|   string|   NULL|
|         MPG|   string|   NULL|
|   Cylinders|   string|   NULL|
|Displacement|   string|   NULL|
|  Horsepower|   string|   NULL|
|      Weight|   string|   NULL|
|Acceleration|   string|   NULL|
|       Model|   string|   NULL|
|      Origin|   string|   NULL|
+------------+---------+-------+



In [7]:
# Casting MPG column to double datatype and checking
spark.sql("SELECT CAST(MPG AS DOUBLE) AS MPG_double, * FROM cars").show()
spark.sql("SELECT CAST(MPG AS DOUBLE) AS MPG_double, * FROM cars").printSchema()

+----------+--------------------+----+---------+------------+----------+------+------------+-----+------+
|MPG_double|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+----------+--------------------+----+---------+------------+----------+------+------------+-----+------+
|      18.0|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|      15.0|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|      18.0|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|      16.0|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|      17.0|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|      15.0|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|      14.0|    Chevrolet Impala|14.0|        

In [13]:
# Adding array column
spark.sql("SELECT *, ARRAY(Horsepower, Weight) AS hp_weight FROM cars").show()
spark.sql("SELECT *, ARRAY(Horsepower, Weight) AS hp_weight FROM cars").printSchema()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|     hp_weight|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|[130.0, 3504.]|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|[165.0, 3693.]|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|[150.0, 3436.]|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|[150.0, 3433.]|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|[140.0, 3449.]|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|[198.0, 4341.]|
|    Chevr


# Literals

In [12]:
# Adding literal in Origin column
spark.sql("SELECT *, CONCAT(origin, '_Origin') AS country FROM cars").show()
spark.sql("SELECT *, CONCAT(origin, '_Origin') AS country FROM cars").printSchema()

+--------------------+----+---------+------------+----------+------+------------+-----+------+-------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|      country|
+--------------------+----+---------+------------+----------+------+------------+-----+------+-------------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|    US_Origin|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|    US_Origin|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|    US_Origin|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|    US_Origin|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|    US_Origin|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|    US_Origin|
|    Chevrolet Impa

# Data Defintion Statements(DDL)

In [19]:
# 1. Create Database
spark.sql("CREATE DATABASE IF NOT EXISTS mycar")

DataFrame[]

In [20]:
# 2. Show all databases
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
|   my_car|
|    mycar|
+---------+



In [14]:
# 3. Use my_car databases
spark.sql("USE DATABASE my_car").show()

++
||
++
++



In [21]:
# 4. Create table
spark.sql("""
  CREATE TABLE alcars (name STRING, MPG DOUBLE) USING CSV
""").show()

++
||
++
++



In [22]:
# 5. Show tables
spark.sql("SHOW TABLES").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   my_car|   alcars|      false|
|   my_car| all_cars|      false|
|         |     cars|       true|
+---------+---------+-----------+





---

